## Vídeo 1.2 - Entendendo o problema

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/allanspadini/dados-com-muitas-dimensoes/main/dados/hotel.csv')

In [ ]:
df

In [ ]:
df['booking_status'].unique()

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.info()

## Vídeo 1.3 - Treinando um modelo de ML

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modelo = RandomForestClassifier(random_state=2)

X = df.drop(['booking_status'],axis=1)
y = df['booking_status']

X_treino,X_teste, y_treino, y_teste = train_test_split(X,y, random_state=50, test_size=0.3,stratify=y)


In [ ]:
y_teste.shape

In [ ]:
modelo.fit(X_treino,y_treino)
modelo.score(X_teste,y_teste)

In [ ]:
(df['booking_status'].value_counts() / len(df))*100

# Aula 2 - Análise de dados

## Vídeo 2.1 - Entendendo as features

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(X_treino, x=y_treino, y='arrival_month')

In [ ]:
sns.violinplot(X_treino, x=y_treino, y='arrival_month')

In [ ]:
sns.boxplot(X_treino, x=y_treino, y='lead_time')

## Vídeo 2.2 - Correlação entre features

In [ ]:
correlacao = pd.concat([X_treino,y_treino],axis=1).corr()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,8))
sns.heatmap(correlacao, annot=True, cmap='coolwarm',fmt='.2f')

## Vídeo 2.3 - Feature Importance

In [ ]:
modelo = RandomForestClassifier(random_state=2)
modelo.fit(X_treino,y_treino)
print(f'Score: {modelo.score(X_teste,y_teste)}')
importancia = pd.DataFrame({'colunas':X_treino.columns,'importancia':modelo.feature_importances_})

In [ ]:
importancia10 = importancia.sort_values(by="importancia", ascending=False).head(10)

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.barplot(x='colunas',y='importancia',data=importancia10)
ax.set_xticklabels(ax.get_xticklabels(), rotation=75);

# Aula 3 - Automatizando a seleção

## Vídeo 3.1 - SelectFromModel

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
modelo_fselecionadas = SelectFromModel(modelo, max_features=10, prefit=True)

In [ ]:
X_treino_selecionado = modelo_fselecionadas.transform(X_treino);
X_teste_selecionado = modelo_fselecionadas.transform(X_teste);

In [ ]:
modelo = RandomForestClassifier(random_state=2)
modelo.fit(X_treino_selecionado,y_treino)
print(f'Score: {modelo.score(X_teste_selecionado,y_teste)}')

## Vídeo 3.2 - Validação cruzada

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
modelo = RandomForestClassifier(random_state=2)

In [ ]:
parametros_busca = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [ ]:
grid_search = GridSearchCV(estimator=modelo, param_grid=parametros_busca,
                           scoring='accuracy', cv=5, n_jobs=-1)

In [ ]:
grid_search.fit(X_treino, y_treino)

Ou utilizar direto o melhor estimador na SelectFromModel.

In [ ]:
modelo_fselecionadas = SelectFromModel(grid_search.best_estimator_, max_features=10,prefit=True)


In [ ]:
X_treino_selecionado = modelo_fselecionadas.transform(X_treino)
X_teste_selecionado = modelo_fselecionadas.transform(X_teste)

In [ ]:
modelo = grid_search.best_estimator_
modelo.fit(X_treino_selecionado, y_treino)

In [ ]:
score = modelo.score(X_teste_selecionado, y_teste)
print(f'Score: {score}')

## Vídeo 3.3 - Matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

In [ ]:
y_pred = modelo.predict(X_teste_selecionado)

In [ ]:
matriz_confusao = confusion_matrix(y_teste,y_pred)

sns.heatmap(matriz_confusao, annot=True, cmap='coolwarm',fmt='d')
plt.xlabel('Classe Prevista');
plt.ylabel('Classe Real');
plt.title('Matriz de Confusão');


Outro método de análise que podemos usar é a curva roc.

In [ ]:
fpr, tpr, thresholds = roc_curve(y_teste,y_pred)

In [ ]:
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (área = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

# Aula 4 - Avaliação dos resultados

## Vídeo 4.1 - Seleção com RFE

In [ ]:
grid_search.best_params_

In [ ]:
parametros = {
    'n_estimators': 300,
    'max_depth': None,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'random_state': 2
}

In [ ]:
modelo = RandomForestClassifier(**parametros)

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
selecionador = RFE(modelo,n_features_to_select=10, step=1)
selecionador.fit(X_treino, y_treino)

In [ ]:
X_treino_selecionado = selecionador.transform(X_treino)
X_teste_selecionado = selecionador.transform(X_teste)

In [ ]:
modelo.fit(X_treino_selecionado,y_treino)
print(f'Score: {modelo.score(X_teste_selecionado,y_teste)}')

In [ ]:
modelo.fit(X_treino_selecionado,y_treino)

In [ ]:
y_pred = modelo.predict(X_teste_selecionado)

In [ ]:
matriz_confusao = confusion_matrix(y_teste,y_pred)

sns.heatmap(matriz_confusao, annot=True, cmap='coolwarm',fmt='d')
plt.xlabel('Classe Prevista');
plt.ylabel('Classe Real');
plt.title('Matriz de Confusão');

## Vídeo 4.2 - Seleção com RFECV

In [ ]:
modelo = RandomForestClassifier(**parametros)

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

Aqui vamos definir o StratifiedKFold para ser o método de divisão dos dados.

In [ ]:
cv = StratifiedKFold(5,shuffle=True,random_state=500)
rfecv = RFECV(
    estimator=modelo,
    step=1,
    cv=cv,
    scoring='roc_auc', #roc_auc
    min_features_to_select=1,
    importance_getter='auto'
)

In [ ]:
rfecv.fit(X_treino,y_treino)
print(f'O número ideal de features é: {rfecv.n_features_}')
print(f'Ranking das features: {rfecv.ranking_}')
print(f'Score: {rfecv.cv_results_}')

In [ ]:
y_pred = rfecv.predict(X_teste)

In [ ]:
matriz_confusao = confusion_matrix(y_teste,y_pred)

In [ ]:
sns.heatmap(matriz_confusao, annot=True, cmap='coolwarm',fmt='d')

## Vídeo 4.3 - Visualização dos resultados

In [ ]:
resultado = rfecv.cv_results_

In [ ]:
resultado['mean_test_score']

In [ ]:
results_df = pd.DataFrame({'Valores': resultado['mean_test_score']})

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.barplot(x=results_df.index +1 , y='Valores',data=results_df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45);

In [ ]:
resultado['mean_test_score'][27]